# Building a comments classification model

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [2]:
CLASS_NAMES = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

We start with data from Kaggle:

In [3]:
train = pd.read_csv('data/train.csv').fillna(' ')
test = pd.read_csv('data/test.csv').fillna(' ')
test_labels = pd.read_csv('data/test_labels.csv').fillna(' ')

In [4]:
train['non-toxic'] = (~train[CLASS_NAMES].any(axis=1)).astype(int)

In [5]:
train.sample(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,non-toxic
4470,0be8e1ed8ee4aae6,The article reads a lot like a news article. U...,0,0,0,0,0,0,1
125383,9ea57def6058b166,"""\n\nYou are doing unusual things\nI read some...",0,0,0,0,0,0,1
98867,10ea7562b957e332,"""I find it quite bizarre to learn that my vote...",0,0,0,0,0,0,1


In [6]:
train.loc[train['identity_hate'] == 1].sample(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,non-toxic
9090,183a77b265f02ccd,Hey i know that im just a city boy but this pa...,1,0,1,0,1,1,0
127419,a97eabb625eb4f37,Rklaton is faggot and loves to suck cock and g...,1,0,1,0,1,1,0
43233,7366e2600f2ea082,"Fuck Wikipedia and fuck all the leftist, liber...",1,1,1,0,1,1,0


In [7]:
train.shape

(159571, 9)

In [8]:
test.sample(3)

,id,comment_text
79275,844a2931f316bd0d,"I will kill you, I know where you live..."
28351,2f2dfec8fce34be5,== Rackmbo == \n\n Rocky drives a car? no big ...
30868,3355cd4e13c248fb,== What the heck?! == \n\n How in the freaking...


In [9]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text], axis=0)

Fit a word vectorizer on all the comments we have at hand.

In [10]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000
)
word_vectorizer.fit(all_text)

train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

KeyboardInterrupt: 

Fit a character vectorizer on all the comments we have at hand.

In [ ]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000
)
char_vectorizer.fit(all_text)

train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

Combine char and word features.

In [ ]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

Perform cross-validation.

In [ ]:
models = {}
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in CLASS_NAMES:
    train_target = train[class_name]
    classifier = LogisticRegressionCV(Cs=[0.1], cv=5, dual=False, scoring='roc_auc', solver='sag')
    classifier.fit(train_features, train_target)
    models[class_name] = classifier


In [ ]:
import pickle
import sklearn
print(pickle.__version__)
print(sklearn.__version__)
with open('comments_clf_models.pkl', 'wb') as f:
    pickle.dump(models, f)